In [1]:
import tensorflow as tf
with tf.name_scope('Model0') as scope:
    with tf.variable_scope("Add1"):
        a = tf.get_variable('a', [1], dtype=tf.int32, initializer=tf.zeros_initializer)
        c = tf.constant(20, dtype=tf.int32, name="c")
        print(c.op.name)
        print(a.op.name)

/home/dragonx/venv3/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Model0/Add1/c
Add1/a


In [2]:
b = a+c

In [3]:
print(b.op.name)

add


In [4]:
d = b+a
print(d.op.name)

add_1


In [5]:
g = tf.get_default_graph()
print(g.get_operations())
print(g.get_all_collection_keys)

[<tf.Operation 'Add1/a/Initializer/zeros' type=Const>, <tf.Operation 'Add1/a' type=VariableV2>, <tf.Operation 'Add1/a/Assign' type=Assign>, <tf.Operation 'Add1/a/read' type=Identity>, <tf.Operation 'Model0/Add1/c' type=Const>, <tf.Operation 'add' type=Add>, <tf.Operation 'add_1' type=Add>]
<bound method Graph.get_all_collection_keys of <tensorflow.python.framework.ops.Graph object at 0x7f325af02358>>


In [6]:
tf.reset_default_graph()
g = tf.get_default_graph()
print(g.get_operations())

[]


In [7]:
# build model 
with tf.name_scope('Model1') as scope:
    with tf.variable_scope('feature_fusion'):
        a = tf.get_variable('a', shape=[1], dtype=tf.float32, initializer=tf.ones_initializer)
        b = tf.constant(20.0, dtype=tf.float32, name='b')
c = a + b
print(a.op.name, b.op.name, c.op.name)






feature_fusion/a Model1/feature_fusion/b add


In [21]:
# ops to save and restore model
saver = tf.train.Saver()
# ops to initialize the variables
init_op = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init_op)
    value = sess.run(c)
    print(value)
    save_path = saver.save(sess, './model.ckpt')
    print("Model saved in path: %s" %save_path)

[21.]
Model saved in path: ./model.ckpt


In [23]:
saver

In [49]:
# restore a model 
tf.reset_default_graph()
with tf.name_scope('Model2') as scope:
    with tf.variable_scope('Step1'):
        with tf.variable_scope('feature_fusion'):
            a = tf.get_variable('a', shape=[1])
            print(a.op.name)
    with tf.variable_scope('feature_fusion'):
        a_1 = tf.get_variable('a', shape=[1])
        print(a_1.op.name)   
# here we must rebuild saver ops
# but we would like to restore variables by 
saver = tf.train.Saver({a_1.op.name:a_1})
with tf.Session() as sess:
    # restore
    saver.restore(sess, './model.ckpt')
    print("a: %s" % a_1.eval())


Step1/feature_fusion/a
feature_fusion/a
INFO:tensorflow:Restoring parameters from ./model.ckpt
a: [1.]


In [50]:
# restore a model 
tf.reset_default_graph()
with tf.name_scope('Model2') as scope:
    with tf.variable_scope('Step1'):
        with tf.variable_scope('feature_fusion'):
            a = tf.get_variable('a', shape=[1])
            print(a.op.name)
# here we must rebuild saver ops
# but we would like to restore variables by 
saver = tf.train.Saver({'feature_fusion/a':a})
with tf.Session() as sess:
    # restore
    saver.restore(sess, './model.ckpt')
    print("a: %s" % a.eval())
    print(a.op.name)


Step1/feature_fusion/a
INFO:tensorflow:Restoring parameters from ./model.ckpt
a: [1.]
Step1/feature_fusion/a


In [83]:
# restore a model 
tf.reset_default_graph()
with tf.name_scope('Model2'):
    with tf.variable_scope('Step1'):
        with tf.variable_scope('feature_fusion') as scope:
            a = tf.get_variable('a', shape=[1])
            scope.reuse_variables()
            a_1 = tf.get_variable('a', shape=[1])
            print(a.op.name)
            print(a_1.op.name)
# here we must rebuild saver ops
# but we would like to restore variables by 
vars = tf.contrib.framework.list_variables('.')
print(vars)
# new_vars = []
# for name, shape in vars:
#     v = tf.contrib.framework.load_variable('.', name)
#     new_vars.append(tf.Variable(v, name=name.replace('Step1/', '')))
saver = tf.train.Saver({v.op.name.replace('Step1/', ''): v for v in tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)})
# init_op = tf.global_variables_initializer()
with tf.Session() as sess:
    # sess.run(init_op)
    # restore
    saver.restore(sess, './model.ckpt')
    print("a: %s" % a.eval())
    print("a: %s" % a_1.eval())
    print(a.op.name)

Step1/feature_fusion/a
Step1/feature_fusion/a
[('feature_fusion/a', [1])]
INFO:tensorflow:Restoring parameters from ./model.ckpt
a: [1.]
a: [1.]
Step1/feature_fusion/a


In [100]:
# restore a model 
import tensorflow.contrib.slim as slim
tf.reset_default_graph()
with tf.name_scope('Model2'):
    with tf.variable_scope('Step1'):
        with tf.variable_scope('feature_fusion') as scope:
            a = tf.get_variable('a', shape=[1])
            scope.reuse_variables()
            a_1 = tf.get_variable('a', shape=[1])
            print(a.op.name)
            print(a_1.op.name)
# here we must rebuild saver ops
# but we would like to restore variables by 
vars = tf.contrib.framework.list_variables('.')
print(tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES))
print(slim.get_trainable_variables())
for v in tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES):
    print(v.op.name[len(v.op.name.split('/')[0])+1:])
# new_vars = []
# for name, shape in vars:
#     v = tf.contrib.framework.load_variable('.', name)
#     new_vars.append(tf.Variable(v, name=name.replace('Step1/', '')))
saver = tf.train.Saver({v.op.name[len(v.op.name.split('/')[0])+1:]: v for v in tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)})
# init_op = tf.global_variables_initializer()
with tf.Session() as sess:
    # sess.run(init_op)
    # restore
    saver.restore(sess, './model.ckpt')
    print("a: %s" % a.eval())
    print("a: %s" % a_1.eval())
#     print(a.op.name)

Step1/feature_fusion/a
Step1/feature_fusion/a
[<tf.Variable 'Step1/feature_fusion/a:0' shape=(1,) dtype=float32_ref>]
[<tf.Variable 'Step1/feature_fusion/a:0' shape=(1,) dtype=float32_ref>]
feature_fusion/a
INFO:tensorflow:Restoring parameters from ./model.ckpt
a: [1.]
a: [1.]


In [70]:
# here we have two ways to solve this problem
# first let's do an inspection over the variables
from tensorflow.python.tools import inspect_checkpoint as chkp
# print all tensors in checkpoint file
chkp.print_tensors_in_checkpoint_file("./model.ckpt", tensor_name='', all_tensors=True)
vars = tf.contrib.framework.list_variables('.')
print(vars)
new_vars = []
for name, shape in vars:
    v = tf.contrib.framework.load_variable('.', name)
    new_vars.append(tf.Variable(v, name='Step1/' + name))
print(new_vars)

tensor_name:  feature_fusion/a
[1.]
[('feature_fusion/a', [1])]
[<tf.Variable 'Step1/feature_fusion/a_2:0' shape=(1,) dtype=float32_ref>]


In [39]:
print(tf.model_variables())
print(tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES))
print(tf.trainable_variables())

[]
[<tf.Variable 'feature_fusion/a:0' shape=(1,) dtype=float32_ref>]
[<tf.Variable 'feature_fusion/a:0' shape=(1,) dtype=float32_ref>]
